In [1]:
"""
This Script pulls the baseline data for our project. This data is then used for our baseline linear model outlined in: 
1. Stock returns and firm characteristics (inspired by the Fama French 5-factor model) are pulled from the Global Factor Database in WRDS (Wharton Research Data Services) 
    which combines data from the survivor bias free stock return dataset CRSP (Centre for Research in Securities Prices) with public company financial information from Compustat.
2. Macro variables from FRED including the slope of the yield curve, short and long term interest rates, and the unemployment rate
"""

import wrds
import pandas as pd
from fredapi import Fred

## Connect to WRDS
wrds_db = wrds.Connection()
## Get Ben's FRED api key
fred = Fred(api_key_file="C:/Users/biorio/Desktop/fredkey.txt")

Loading library list...
Done


In [2]:
## Documentation on all of these characteristics can be found here: https://jkpfactors.s3.amazonaws.com/documents/Documentation.pdf

# Take characteristics associated with fama-french factors - Size, B/M, Operating Profit, Asset Growth, Momentum (1-3 months - "randomly" chosen)
sql_query = f""" 
    SELECT id, eom, excntry, gvkey, permno, prc_local, ret_exc, ret_exc_lead1m, me, be_me, ope_be, at_gr1, ret_3_1, ff49
    FROM contrib.global_factor
    WHERE common=1 and exch_main=1 and primary_sec=1 and obs_main=1 and
    excntry='USA' and eom > '1985-01-01'
"""

data = wrds_db.raw_sql(sql_query)
data = data.sort_values(['id', 'eom'])
data

,id,eom,excntry,gvkey,permno,prc_local,ret_exc,ret_exc_lead1m,me,be_me,ope_be,at_gr1,ret_3_1,ff49
133904,10000.0,1986-01-31,USA,013007,10000.0,4.375,<NA>,-0.262439,16.1,<NA>,<NA>,<NA>,<NA>,<NA>
133905,10000.0,1986-02-28,USA,013007,10000.0,3.25,-0.262439,0.359422,11.96,<NA>,<NA>,<NA>,<NA>,<NA>
133906,10000.0,1986-03-31,USA,013007,10000.0,4.4375,0.359422,-0.103933,16.33,<NA>,<NA>,<NA>,-0.257143,<NA>
133907,10000.0,1986-04-30,USA,013007,10000.0,4.0,-0.103933,-0.22759,15.172,<NA>,<NA>,<NA>,0.014286,<NA>
133908,10000.0,1986-05-31,USA,013007,10000.0,3.109375,-0.22759,-0.010271,11.793859,0.05842,-0.763425,<NA>,0.230769,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199818,135628901.0,2024-01-31,USA,356289,<NA>,6.355,<NA>,<NA>,170.676235,0.082321,-0.120398,<NA>,<NA>,<NA>
200058,135628901.0,2024-02-29,USA,356289,<NA>,5.0,<NA>,<NA>,134.285,0.10463,-0.120398,<NA>,<NA>,<NA>
200551,135628901.0,2024-03-31,USA,356289,<NA>,5.29,<NA>,<NA>,143.54944,0.097878,-0.120398,<NA>,<NA>,<NA>
211180,135628901.0,2025-01-31,USA,356289,<NA>,2.72,<NA>,<NA>,84.32544,0.536583,-0.009943,5.006107,<NA>,<NA>


In [3]:
data = data.sort_values(['id', 'eom'])

data.to_csv('base_variables.csv.gz', compression="gzip", index=False) 

In [7]:
## Macro Variables from FRED
## Pull 10Y rate, yield curve slope, and Fed Funds Rate (to use as Risk free)
series_ids = ['FEDFUNDS', 'T10Y2Y', 'BAA10Y', 'VIXCLS', 'INDPRO'] # 'DGS10', 
df = pd.DataFrame()
for series_id in series_ids:
    s = fred.get_series(series_id)
    s.name = series_id # Assign the series ID as the column name
    if df.empty:
        df = s.to_frame() # Convert the first series to a DataFrame
    else:
        df = df.join(s, how='outer') # Join subsequent series

## aggregate to monthly
df = df[df.index>='1985-01-01'].resample('M').last()
df = df.sort_index()

## take % changes of industrial production so all variables are stationary, and lag one month for lookahead bias
df['INDPRO'] = df['INDPRO'].pct_change(fill_method=None).shift(1)


df = df.reset_index(names = 'date')
df

C:\Users\biorio\AppData\Local\Temp\ipykernel_27316\1854778215.py:14: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df = df[df.index>='1985-01-01'].resample('M').last()


,date,FEDFUNDS,T10Y2Y,BAA10Y,VIXCLS,INDPRO
0,1985-01-31,8.35,1.27,NaN,NaN,NaN
1,1985-02-28,8.50,1.25,NaN,NaN,NaN
2,1985-03-31,8.58,1.22,NaN,NaN,0.002936
3,1985-04-30,8.27,1.50,NaN,NaN,0.001467
4,1985-05-31,7.97,1.36,NaN,NaN,-0.002202
...,...,...,...,...,...,...
486,2025-07-31,4.33,0.43,1.67,16.72,0.005349
487,2025-08-31,4.33,0.64,1.80,15.36,-0.003763
488,2025-09-30,4.22,0.56,1.67,16.28,0.000972
489,2025-10-31,4.09,0.51,1.69,17.44,NaN


In [8]:
df.to_csv('macro_variables.csv.gz', compression="gzip", index=False) 